In [74]:
# load package# load p 
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation  import train_test_split
from sklearn import metrics
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn import cross_validation

import ffm
import _pickle as pickle

from fastFM.mcmc import FMClassification, FMRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.datasets import dump_svmlight_file
# set path
import os
default_path = "/Users/francislin/jrml/Recommendation/"
os.chdir(default_path)

# skip interaction warning
import warnings; 
warnings.filterwarnings('ignore')

In [26]:
# Data Prepararion
# read data
#train=pd.read_csv('train.csv',encoding='big5')
class0=pd.read_csv('WORK_MAX_ROUND_CLASS0_ALL.csv',encoding='big5')
class1=pd.read_csv('ROUND_CLASS1_ALL.csv',encoding='big5')
train= class0.append([class1,class1,class1,class1], ignore_index=True)#放大4倍Y=1
test=pd.read_csv('test_public.csv',encoding='big5')
submit=pd.read_csv('sampleSubmission.csv',encoding='big5')
data=train.append(test, ignore_index=True)

In [27]:
# 運用'Class'欄位來判斷訓練集還是測試集
dataTrain = data[pd.notnull(data['Class'])].sort_values(by=["Time"])
dataTest = data[~pd.notnull(data['Class'])].sort_values(by=["Time"])
dataTrain.columns

Index(['Amount', 'Class', 'TXKEY', 'Time', 'V1', 'V10', 'V11', 'V12', 'V13',
       'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V2', 'V20', 'V21', 'V22',
       'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V3', 'V4', 'V5', 'V6', 'V7',
       'V8', 'V9'],
      dtype='object')

In [63]:
from sklearn.model_selection import train_test_split
train, test= train_test_split(dataTrain, test_size=0.3, random_state=7)

In [56]:
# Based on Kaggle kernel by Scirpus
def convert_to_ffm(df,type,numerics,categories,features):
    currentcode = len(numerics)
    catdict = {}
    catcodes = {}
    # Flagging categorical and numerical fields
    for x in numerics:
         catdict[x] = 0
    for x in categories:
         catdict[x] = 1
    
    nrows = df.shape[0]
    ncolumns = len(features)
    with open(str(type) + "_ffm.txt", "w") as text_file:
# Looping over rows to convert each row to libffm format
        for n,r in enumerate(range(nrows)):
            datastring = ""
            datarow = df.iloc[r].to_dict()
            #datastring += str(int(datarow['Class']))
            datastring += str(int(datarow['Class']))
             # For numerical fields, we are creating a dummy field here
            for i, x in enumerate(catdict.keys()):
                if(catdict[x]==0):
                    datastring = datastring + " "+str(i)+":"+ str(i)+":"+ str(datarow[x])
                else:
            # For a new field appearing in a training example
                    if(x not in catcodes):
                        catcodes[x] = {}
                        currentcode +=1
                        catcodes[x][datarow[x]] = currentcode #encoding the feature
            # For already encoded fields
                    elif(datarow[x] not in catcodes[x]):
                        currentcode +=1
                        catcodes[x][datarow[x]] = currentcode #encoding the feature
                    code = catcodes[x][datarow[x]]
                    datastring = datastring + " "+str(i)+":"+ str(int(code))+":1"

            datastring += '\n'
            text_file.write(datastring)

In [67]:
num_col_tr = train.iloc[:,4:32]
print (num_col_tr.head())
print ('\n')

num_col_te = test.iloc[:,4:32]
print (num_col_tr.head())
print ('\n')

         V1  V10  V11  V12  V13  V14  V15  V16  V17  V18 ...   V26  V27  V28  \
156699  1.9  1.3 -0.3  0.2  1.6 -1.0  0.8 -1.1  1.1 -1.3 ...  -0.2  0.1  0.0   
44105  -1.0  0.0 -1.3 -1.0 -1.5 -1.4 -1.5  0.4  1.0 -1.6 ...  -0.1  0.5  0.2   
71662  -0.5 -0.5  0.0 -0.4 -1.8 -0.6 -1.3  0.0  0.6  1.1 ...   0.4  0.3  0.4   
60097  -0.7  0.7  0.0 -0.2  1.4 -0.1 -0.8  0.3 -0.2  0.7 ...  -0.4  0.0  0.0   
125873  1.1 -0.5  2.0  0.0 -1.6 -0.8  0.8  0.5  1.1  0.7 ...  -0.3  0.0  0.0   

         V3   V4   V5   V6   V7   V8   V9  
156699  0.0 -1.5 -1.5  1.1 -1.9  0.4 -0.7  
44105   2.9 -1.5 -1.4  0.6 -0.9  0.6  0.5  
71662  -1.2  0.1  2.4 -1.1 -0.3  0.2  0.4  
60097   0.8 -2.7 -0.5 -0.1 -0.4  0.4 -3.0  
125873 -0.3  1.2  0.1 -0.6  0.0  0.1 -0.2  

[5 rows x 28 columns]


         V1  V10  V11  V12  V13  V14  V15  V16  V17  V18 ...   V26  V27  V28  \
156699  1.9  1.3 -0.3  0.2  1.6 -1.0  0.8 -1.1  1.1 -1.3 ...  -0.2  0.1  0.0   
44105  -1.0  0.0 -1.3 -1.0 -1.5 -1.4 -1.5  0.4  1.0 -1.6 ...  -0.1  0.

In [75]:
num_col_tr = pd.DataFrame(num_col_tr.fillna(num_col_tr.mean()))
cat_col_tr = pd.DataFrame()
all_col_tr = pd.concat([num_col_tr,cat_col_tr],axis=1)

In [76]:
train_data_Label = pd.concat([train.Class, all_col_tr],axis=1)
convert_to_ffm(train_data_Label,'CC_Train',list(num_col_tr),list(cat_col_tr),list(all_col_tr))

In [77]:
%pwd

'/Users/francislin/JrML/Recommendation'

In [78]:
num_col_te = pd.DataFrame(num_col_te.fillna(num_col_tr.mean()))
cat_col_te = pd.DataFrame()
all_col_te = pd.concat([num_col_te,cat_col_te],axis=1)

In [79]:
test_data_Label = pd.concat([test.Class, all_col_te],axis=1)
convert_to_ffm(test_data_Label,'CC_Test',list(num_col_te),list(cat_col_te),list(all_col_te))

In [221]:
#開始針對真正要預測的資料做整理
num_col_rt = dataTest.iloc[:,4:32]
print (num_col_rt.head())
print ('\n')
num_col_rt = pd.DataFrame(num_col_rt.fillna(num_col_rt.mean()))
cat_col_rt = pd.DataFrame()
all_col_rt = pd.concat([num_col_rt,cat_col_rt],axis=1)

              V1       V10       V11       V12       V13       V14       V15  \
188341 -1.358354  0.207643  0.624501  0.066084  0.717293 -0.165946  2.345865   
188342 -0.425966 -0.371407  1.341262  0.359894 -0.358091 -0.137134  0.517617   
188343 -0.644269  1.249376 -0.619468  0.291474  1.757964 -1.323865  0.686133   
188344 -0.338262 -0.366846  1.017614  0.836390  1.006844 -0.443523  0.150219   
188345  1.449044  1.626659  1.199644 -0.671440 -0.513947 -0.095045  0.230930   

             V16       V17       V18    ...          V26       V27       V28  \
188341 -2.890083  1.109969 -0.121359    ...    -0.139097 -0.055353 -0.059752   
188342  0.401726 -0.058133  0.068653    ...     0.105915  0.253844  0.081080   
188343 -0.076127 -1.222127 -0.358222    ...    -0.051634 -1.206921 -1.085339   
188344  0.739453 -0.540980  0.476677    ...     0.094199  0.246219  0.083076   
188345  0.031967  0.253415  0.854344    ...    -0.129478  0.042850  0.016253   

              V3        V4        V5  

In [226]:
r_test_data_Label = pd.concat([dataTest.Class, all_col_rt],axis=1)
dataTest['Class']=0 #先設定0，無法一開始NaN，Cuz ValueError: cannot convert float NaN to integer
convert_to_ffm(r_test_data_Label,'r_Test',list(num_col_rt),list(cat_col_rt),list(all_col_rt))

In [80]:
import xlearn as xl

### FFM

In [246]:
# Training task
ffm_model = xl.create_ffm() # Use field-aware factorization machine
ffm_model.setTrain("CC_Train_ffm.txt")  # Training data
ffm_model.setValidate("CC_Test_ffm.txt")  # Validation data

# param:
#  0. binary classification
#  1. learning rate: 0.2
#  2. regular lambda: 0.002
#  3. evaluation metric: accuracy
param = {'task':'binary', 'lr':0.2, 
         'lambda':0.002, 'metric':'f1'}

# Start to train
# The trained model will be stored in model.out
ffm_model.fit(param, './model.out')

# Prediction task
ffm_model.setTest("CC_Test_ffm.txt")  # Test data
ffm_model.setSigmoid()  # Convert output to 0-1

# Start to predict
# The output result will be stored in output.txt
ffm_model.predict("./model.out", "./output_ffm.txt")

In [228]:
output_ffm =pd.read_csv('./output_ffm.txt',encoding='big5',header=None)
output_ffm.reset_index(drop=True, inplace=True)
t=test[['TXKEY','Class']]
t.reset_index(drop=True, inplace=True)
test_result= pd.concat([t, output_ffm], axis=1)
test_result= test_result.sort_values(by=[0], ascending=False) #descending rate
test_result = test_result.rename(columns={0:'rate'})
test_result['C']=0
test_result.C[test_result.rate > test_result.iloc[300,2]]=1.0 #前251高的設為1
test_result['check']=test_result['C']+test_result['Class']
test_result.head()

,TXKEY,Class,rate,C,check
19868,T9487,1.0,0.082368,1,2.0
24413,T9509,1.0,0.082326,1,2.0
30747,T9509,1.0,0.082326,1,2.0
52168,T9252,1.0,0.081887,1,2.0
50720,T9035,1.0,0.081548,1,2.0


In [229]:
tw_y_test = test_result['Class']
tw_y_predict = test_result['C']
from sklearn import metrics
print(metrics.classification_report(tw_y_test, tw_y_predict))
print(metrics.confusion_matrix(tw_y_test, tw_y_predict))

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00     56109
        1.0       0.95      0.72      0.82       394

avg / total       1.00      1.00      1.00     56503

[[56093    16]
 [  110   284]]


In [247]:
# Prediction task
ffm_model.setTest("r_Test_ffm.txt")  # Test data
ffm_model.setSigmoid()  # Convert output to 0-1

# Start to predict
# The output result will be stored in output.txt
ffm_model.predict("./model.out", "./r_Test_output_ffm.txt")

In [248]:
final_output_ffm =pd.read_csv('./r_Test_output_ffm.txt',encoding='big5',header=None)
final_output_ffm.reset_index(drop=True, inplace=True)
t=dataTest[['TXKEY','Class']]
t.reset_index(drop=True, inplace=True)
final_result= pd.concat([t, final_output_ffm], axis=1)

In [249]:
final_result= final_result.sort_values(by=[0], ascending=False) #descending rate
final_result = final_result.rename(columns={0:'rate'})
final_result['C']=0
final_result.C[final_result.rate > final_result.iloc[158,2]]=1.0 #前158高的設為1

In [250]:
final_result['Class']=final_result['C']
submit=final_result[['TXKEY','Class']]

In [251]:
submit.to_csv('./CCTXN_to_kaggle_FM.csv', index= False)
%pwd

'/Users/francislin/JrML/Recommendation'

### FM

In [82]:
fm_model = xl.create_fm() # Use field-aware factorization machine
fm_model.setTrain("CC_Train_ffm.txt")  # Training data
fm_model.setValidate("CC_Test_ffm.txt")  # Validation data
# param:
#  0. binary classification
#  1. learning rate : 0.2
#  2. regular lambda : 0.002
param = {'task':'binary', 'lr':0.02, 'lambda':0.002,  'metric':'acc'}
# Train model
fm_model.fit(param, "./model_fm.out")

# Prediction task
fm_model.setTest("CC_Test_ffm.txt")  # Test data
fm_model.setSigmoid()  # Convert output to 0-1

# Start to predict
# The output result will be stored in output.txt
fm_model.predict("./model_fm.out", "./output_fm.txt")

In [165]:
output_fm =pd.read_csv('./output_fm.txt',encoding='big5',header=None)

In [252]:
output_fm=output_fm.sort_values(by=[0], ascending=False)